In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [137]:
import sys
sys.path.append('../src')
from satellite_image import SatelliteImage
from utils import *
from plot_utils import *
from mappings import *
from labeler import (RIL_BDTOPOLabeler, BDTOPOLabeler, RILLabeler)
from labeled_satellite_image import SegmentationLabeledSatelliteImage

In [5]:
import s3fs
import matplotlib.pyplot as plt
import re

In [6]:
update_storage_access()

environment = get_environment()

root_path = get_root_path()

bucket = environment["bucket"]

path_s3_pleiades_data_2022_guyane = environment["sources"]["PLEIADES"][2022]["guyane"]
path_local_pleiades_data_2022_guyane = environment["local-path"]["PLEIADES"][2022]["guyane"]

path_local_bdtopo_data_guyane = environment["local-path"]["BDTOPO"][2022]["guyane"]
path_s3_bdtopo_data_guyane = environment["sources"]["BDTOPO"][2022]["guyane"]

#martinique 2022
path_s3_pleiades_data_2022_martinique = environment["sources"]["PLEIADES"][2022]["martinique"]
path_local_pleiades_data_2022_martinique = environment["local-path"]["PLEIADES"][2022]["martinique"]

path_local_bdtopo_data_martinique = environment["local-path"]["BDTOPO"][2022]["martinique"]
path_s3_bdtopo_data_martinique = environment["sources"]["BDTOPO"][2022]["martinique"]

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})



In [122]:
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2022_guyane}",
        lpath=f"../{path_local_pleiades_data_2022_guyane}",
        recursive=True)

fs.download(
        rpath=f"{bucket}/{path_s3_bdtopo_data_guyane}",
        lpath=f"../{path_local_bdtopo_data_guyane}",
        recursive=True
)

In [7]:
# DL PLEIADE martinique 2022
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data_2022_martinique}",
        lpath=f"../{path_local_pleiades_data_2022_martinique}",
        recursive=True)

In [8]:
fs.download(
        rpath=f"{bucket}/{path_s3_bdtopo_data_martinique}",
        lpath=f"../{path_local_bdtopo_data_martinique}",
        recursive=True
)

In [132]:
#mont baduel
from image_utils import *
find_image_of_point([4.918313, -52.303234], '../data/PLEIADES/2022/GUYANE', coord_gps = True)

In [148]:
#filename = '../data/PLEIADES/2022/GUYANE/ORT_2022072050325085_0353_0545_U22N_16Bits.jp2'
filename = '../data/PLEIADES/2022/GUYANE/ORT_2022072050325085_0355_0544_U22N_16Bits.jp2'
date2022 = date.fromisoformat('2022-01-01')

In [149]:
image = SatelliteImage.from_raster(
        filename,
        date = date2022, 
        n_bands = 4,
        dep = '973'
    )
image.normalize()

In [150]:
labeler_RIL = RILLabeler(date2022, '973')

In [151]:
mask_RIL = labeler_RIL.create_segmentation_label(image)

In [152]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image.array, (1, 2, 0))[:,:,:3])
ax.imshow(mask_RIL, alpha=0.3)

In [153]:
labeler_BDTOPO = BDTOPOLabeler(date2022, '973')

In [219]:
mask_BDTOPO = labeler_BDTOPO.create_segmentation_label(image)

In [155]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image.array, (1, 2, 0))[:,:,:3])
ax.imshow(mask_BDTOPO, alpha=0.3)

In [126]:
labeler_RIL_BDTOPO = RIL_BDTOPOLabeler(date2022, '973')

In [135]:
mask_RIL_BDTOPO = labeler_RIL_BDTOPO.create_segmentation_label(image)

In [136]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image.array, (1, 2, 0))[:,:,:3])
ax.imshow(mask_RIL_BDTOPO, alpha=0.3)

In [9]:
filename_1 = '../data/PLEIADES/2022/MARTINIQUE/ORT_2022_0711_1619_U20N_8Bits.jp2'
date_1 = date.fromisoformat('2022-01-01')

In [70]:
image_1 = SatelliteImage.from_raster(
        filename_1,
        date = date_1, 
        n_bands = 3,
        dep = '972'
    )
image_1.normalize()

In [11]:
image_1.plot([0,1,2])

In [110]:
labeler_BDTOPO_1 = BDTOPOLabeler(date_1, '972')

In [112]:
mask_BDTOPO_1 = labeler_BDTOPO_1.create_segmentation_label(image_1)

In [113]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image_1.array, (1, 2, 0))[:,:,:3])
ax.imshow(mask_BDTOPO_1, alpha=0.3)

In [12]:
labeler_RIL_BDTOPO_1 = RIL_BDTOPOLabeler(date_1, '972')

In [71]:
mask_RIL_BDTOPO_1 = labeler_RIL_BDTOPO_1.create_segmentation_label(image_1)

In [111]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image_1.array, (1, 2, 0))[:,:,:3])
ax.imshow(mask_RIL_BDTOPO_1, alpha=0.3)

In [19]:
from sklearn.cluster import KMeans

In [76]:
from scipy.ndimage import label

In [101]:
import geopandas as gpd
from rasterio.features import rasterize, shapes
from shapely.geometry import Polygon, box

In [114]:
height, width = image_1.array.shape[1:]

# Create a list of polygons from the masked center clouds in order
# to obtain a GeoDataFrame from it
polygon_list = []
for shape in list(shapes(mask_BDTOPO_1)):
    polygon = Polygon(shape[0]["coordinates"][0])
    if polygon.area > 0.85 * height * width:
        continue
    polygon_list.append(polygon)

gdf = gpd.GeoDataFrame(geometry=polygon_list)

In [121]:
# Calculer la taille de chaque polygon
gdf['polygon_size'] = gdf.geometry.area

# Préparer les données pour le clustering
X = gdf['polygon_size'].values.reshape(-1, 1)

# Initialiser l'algorithme de clustering KMeans
kmeans = KMeans(n_clusters=6, random_state=0)

# Appliquer le clustering aux tailles de polygons
kmeans.fit(X)

# Obtenir les étiquettes de cluster pour chaque taille de polygon
labels = kmeans.predict(X)

# Ajouter les étiquettes de cluster à la colonne correspondante du Geodataframe
gdf['cluster'] = labels

# Afficher les polygones avec leur couleur de cluster correspondante
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image_1.array, (1, 2, 0))[:,:,:3])
gdf.plot(column='cluster', cmap='jet', ax=ax)

In [156]:
from sklearn.cluster import AgglomerativeClustering

In [158]:
height, width = image_1.array.shape[1:]

# Create a list of polygons from the masked center clouds in order
# to obtain a GeoDataFrame from it
polygon_list = []
for shape in list(shapes(mask_BDTOPO_1)):
    polygon = Polygon(shape[0]["coordinates"][0])
    if polygon.area > 0.85 * height * width:
        continue
    polygon_list.append(polygon)

gdf = gpd.GeoDataFrame(geometry=polygon_list)

# Calculer la taille de chaque polygon
gdf['polygon_size'] = gdf.geometry.area

# Initialiser l'algorithme de clustering hiérarchique ascendant
agg_clustering = AgglomerativeClustering(n_clusters=4, linkage='ward')

# Appliquer le clustering aux tailles de polygons
agg_clustering.fit(X)

# Obtenir les étiquettes de cluster pour chaque taille de polygon
labels = agg_clustering.labels_

# Ajouter les étiquettes de cluster à la colonne correspondante du Geodataframe
gdf['cluster'] = labels

# Afficher les polygones avec leur couleur de cluster correspondante
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image_1.array, (1, 2, 0))[:,:,:3])
gdf.plot(column='cluster', cmap='jet', ax=ax)

In [159]:
height, width = image_1.array.shape[1:]

# Create a list of polygons from the masked center clouds in order
# to obtain a GeoDataFrame from it
polygon_list = []
for shape in list(shapes(mask_BDTOPO_1)):
    polygon = Polygon(shape[0]["coordinates"][0])
    if polygon.area > 0.85 * height * width:
        continue
    polygon_list.append(polygon)

gdf = gpd.GeoDataFrame(geometry=polygon_list)

# Calculer la taille de chaque polygon
gdf['polygon_size'] = gdf.geometry.area

# Initialiser l'algorithme de clustering hiérarchique descendant
agg_clustering = AgglomerativeClustering(n_clusters=2, linkage='single')

# Appliquer le clustering aux tailles de polygons
agg_clustering.fit(X)

# Obtenir les étiquettes de cluster pour chaque taille de polygon
labels = agg_clustering.labels_

# Ajouter les étiquettes de cluster à la colonne correspondante du Geodataframe
gdf['cluster'] = labels

# Afficher les polygones avec leur couleur de cluster correspondante
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image_1.array, (1, 2, 0))[:,:,:3])
gdf.plot(column='cluster', cmap='jet', ax=ax)

In [163]:
height, width = image_1.array.shape[1:]

# Create a list of polygons from the masked center clouds in order
# to obtain a GeoDataFrame from it
polygon_list = []
for shape in list(shapes(mask_BDTOPO_1)):
    polygon = Polygon(shape[0]["coordinates"][0])
    if polygon.area > 0.85 * height * width:
        continue
    polygon_list.append(polygon)

gdf = gpd.GeoDataFrame(geometry=polygon_list)

# Calculer la taille de chaque polygon
gdf['polygon_size'] = gdf.geometry.area

# Initialiser l'algorithme de clustering hiérarchique descendant
agg_clustering = AgglomerativeClustering(n_clusters=2, linkage='complete')

# Appliquer le clustering aux tailles de polygons
agg_clustering.fit(X)


# Obtenir les étiquettes de cluster pour chaque taille de polygon
labels = agg_clustering.labels_

# Ajouter les étiquettes de cluster à la colonne correspondante du Geodataframe
gdf['cluster'] = labels

# Afficher les polygones avec leur couleur de cluster correspondante
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image_1.array, (1, 2, 0))[:,:,:3])
gdf.plot(column='cluster', cmap='jet', ax=ax)

In [220]:
height, width = image.array.shape[1:]

# Create a list of polygons from the masked center clouds in order
# to obtain a GeoDataFrame from it
polygon_list = []
for shape in list(shapes(mask_BDTOPO)):
    polygon = Polygon(shape[0]["coordinates"][0])
    if polygon.area > 0.85 * height * width:
        continue
    polygon_list.append(polygon)

gdf = gpd.GeoDataFrame(geometry=polygon_list)

In [221]:
# Calculer la taille de chaque polygon
gdf['polygon_size'] = gdf.geometry.area

# Préparer les données pour le clustering
X = gdf['polygon_size'].values.reshape(-1, 1)

# Initialiser l'algorithme de clustering KMeans
kmeans = KMeans(n_clusters=3, n_init = 'auto', random_state=0)

# Appliquer le clustering aux tailles de polygons
kmeans.fit(X)

# Obtenir les étiquettes de cluster pour chaque taille de polygon
labels = kmeans.predict(X)

# Ajouter les étiquettes de cluster à la colonne correspondante du Geodataframe
gdf['cluster'] = labels

gdf['size'] = X

In [176]:
max(gdf[gdf['cluster'] == 0]['size'])

In [222]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image.array, (1, 2, 0))[:,:,:3])
gdf.plot(column='cluster', cmap='jet', ax=ax)

In [182]:
gdf_petite_hab = gdf[gdf['size'] <= 2142.0]

In [302]:
from gestion_label_studio import creer_mask_from_label_studio_export
creer_mask_from_label_studio_export(
    num_task = "1",
    type_label = 'Petites habitations',
    emplacement_zip = "../labelstudio.zip",
    name_output = "bibi"
)



In [183]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image_1.array, (1, 2, 0))[:,:,:3])
gdf_petite_.plot(column='cluster', cmap='jet', ax=ax)

In [185]:
# Rasterize the geometries into a numpy array
if gdf_petite_hab.empty:
    rasterized = np.zeros(image_1.array.shape[1:])
else:
    rasterized = rasterize(
        gdf_petite_hab.geometry,
        out_shape=image_1.array.shape[1:],
        fill=0,
        out=None,
        all_touched=True,
        default_value=1,
        dtype=None,
    )

In [186]:
rasterized

In [187]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image_1.array, (1, 2, 0))[:,:,:3])
ax.imshow(rasterized, alpha=0.3)

In [210]:
max(gdf['polygon_size'])

In [189]:
mask_BDTOPO = labeler_BDTOPO.create_segmentation_label(image)

In [213]:
def BD_TOPO_petite_hab(image, mask_BDTOPO) :
    height, width = image.array.shape[1:]

    # Create a list of polygons from the masked center clouds in order
    # to obtain a GeoDataFrame from it
    polygon_list = []
    for shape in list(shapes(mask_BDTOPO)):
        polygon = Polygon(shape[0]["coordinates"][0])
        # if polygon.area > 0.85 * height * width:
        #     continue
        polygon_list.append(polygon)

    gdf = gpd.GeoDataFrame(geometry=polygon_list)

    # size amas
    gdf['polygon_size'] = gdf.geometry.area
    X = gdf['polygon_size'].values.reshape(-1, 1)
    gdf['size'] = X
    
    #threshold
    gdf_petite_hab = gdf[gdf['polygon_size'] <= 2142.0]
    
    # Rasterize the geometries into a numpy array
    if gdf_petite_hab.empty:
        rasterized = np.zeros(image.array.shape[1:])
    else:
        rasterized = rasterize(
            gdf_petite_hab.geometry,
            out_shape=image.array.shape[1:],
            fill=0,
            out=None,
            all_touched=True,
            default_value=1,
            dtype=None,
        )
    return rasterized

In [214]:
mask_filtred = BD_TOPO_petite_hab(image, mask_BDTOPO)

In [215]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(np.transpose(image.array, (1, 2, 0))[:,:,:3])
ax.imshow(mask_filtred, alpha=0.3)